# **Installing required libraries**

In [ ]:
!pip install torch matplotlib tqdm livelossplot pretty_midi pydub midi2audio

# **Importing Necessary libraries**

In [ ]:
import zipfile
import os
import shutil
import math
import pretty_midi
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pydub import AudioSegment
import tempfile

# **Downloading the maestro dataset**



In [ ]:

dataset_zip_path = 'PATH OF DIRECTORY'
if not os.path.exists(dataset_zip_path):
      !wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip -O {dataset_zip_path}
      print(f"Downloaded MAESTRO dataset to {dataset_zip_path}")
else:
      print(f"MAESTRO dataset already exists at {dataset_zip_path}")

# **Extracting the midi files and organizing**

In [ ]:

# Define paths
dataset_zip_path = 'Path to the downloaded zip file'
extracted_subset_path = 'Directory path for extracted filest'

# Create the target directory
os.makedirs(extracted_subset_path, exist_ok=True)
print(f"Created directory at {extracted_subset_path}")

# Maximum files to extract
max_total_files = 700
total_extracted = 0

# Open the ZIP file
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:

    midi_files = [f for f in zip_ref.namelist() if f.endswith(('.midi', '.mid'))]
    print(f"Found {len(midi_files)} MIDI files in the zip archive.")

    # Group files by year
    files_by_year = {}
    for file in midi_files:
        parts = file.split('/')
        if len(parts) > 1:  # Ensure there's a subdirectory for the year
            year = parts[1]
            files_by_year.setdefault(year, []).append(file)

    num_years = len(files_by_year)
    files_per_year = math.ceil(max_total_files / num_years)

    # Extract files from each year
    for year, files in files_by_year.items():
        extracted_from_year = 0
        for file in files:
            if extracted_from_year < files_per_year and total_extracted < max_total_files:
                # Extract file
                zip_ref.extract(file, 'Base directory path')
                source_path = os.path.join('Base Directory path', file)
                target_path = os.path.join(extracted_subset_path, os.path.basename(file))

                # Move the file to the target directory
                shutil.move(source_path, target_path)
                print(f"Extracted and moved to: {target_path}")

                extracted_from_year += 1
                total_extracted += 1

            if total_extracted >= max_total_files:
                break

        print(f"Extracted {extracted_from_year} files from year {year}.")
        if total_extracted >= max_total_files:
            break

print(f"Extraction complete. Total MIDI files moved: {total_extracted}")


# **Conversion of MIDI files to Pianoroll data**

In [ ]:

# Configuration for calm MIDI to Piano Roll Conversion
n_tracks = 5
n_pitches = 36
lowest_pitch = 36
measure_resolution = 16
n_measures = 4
data = []
# Directory with the 700 extracted files
extracted_subset_path = 'EXTRACTED SUBSET PATH'
def midi_to_piano_roll(midi_file):
    """Convert a MIDI file to a calming piano roll format."""
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_file)
        if not midi_data.instruments:
           print(f"No instruments found in file: {midi_file}")
           return None
        # Extract the piano roll and limit pitch range
        piano_roll = midi_data.instruments[0].get_piano_roll(fs=4 * measure_resolution)
        piano_roll = piano_roll[lowest_pitch:lowest_pitch + n_pitches]
        piano_roll = (piano_roll > 0).astype(np.float32)
        # Trim or pad the piano roll to fit the required length
        max_length = n_measures * measure_resolution
        if piano_roll.shape[1] > max_length:
           piano_roll = piano_roll[:, :max_length]
        else:
          piano_roll = np.pad(piano_roll, ((0, 0), (0, max_length - piano_roll.shape[1])))
        return piano_roll
    except Exception as e:
        print(f"Error processing file {midi_file}: {e}")
        return None
# Process and collect data with calm settings
success_count = 0
failure_count = 0
extracted_files = [f for f in os.listdir(extracted_subset_path) if f.endswith(('.midi', '.mid'))]
print(f"Found {len(extracted_files)} MIDI files in the extracted subset directory.")
for midi_file in extracted_files[:700]: # Limit to the first 700 files
    midi_path = os.path.join(extracted_subset_path, midi_file)
    piano_roll = midi_to_piano_roll(midi_path)
    if piano_roll is not None:
       data.append(piano_roll)
       success_count += 1
       print(f"Successfully converted {success_count} files: {midi_file}")
    else:
        failure_count += 1


data = np.array(data)
print(f"\nData shape after conversion: {data.shape}")
print(f"Total successfully converted files: {success_count}")
print(f"Total failed conversions: {failure_count}")

# **Converting data to a PyTorch tensor**

In [ ]:

data = np.array(data)
data = data.reshape(-1, n_tracks, n_measures * measure_resolution, n_pitches)

data_tensor = torch.as_tensor(data, dtype=torch.float32)
dataset = TensorDataset(data_tensor)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True, drop_last=True)
print("Calm DataLoader created.")
print(f"Data shape for DataLoader: {data_tensor.shape}")

# **Generator and Discriminator Functions**

In [ ]:

# Define required global configuration variables
n_tracks = 5
n_measures = 4
measure_resolution = 16
n_pitches = 36
latent_dim = 128
batch_size = 16
# Generator Block
class GeneraterBlock(nn.Module):
      def __init__(self, in_dim, out_dim, kernel, stride):
          super().__init__()
          self.transconv = nn.ConvTranspose3d(in_dim, out_dim, kernel, stride)
          self.batchnorm = nn.BatchNorm3d(out_dim)
      def forward(self, x):
          x = self.transconv(x)
          x = self.batchnorm(x)
          return torch.relu(x)
# Generator Model
class Generator(nn.Module):
     def __init__(self):
         super().__init__()
         self.transconv0 = GeneraterBlock(latent_dim, 256, (4, 1, 1), (4, 1, 1))
         self.transconv1 = GeneraterBlock(256, 128, (1, 4, 1), (1, 4, 1))
         self.transconv2 = GeneraterBlock(128, 64, (1, 1, 4), (1, 1, 4))
         self.transconv3 = GeneraterBlock(64, 32, (1, 1, 3), (1, 1, 1))
         self.transconv4 = nn.ModuleList([GeneraterBlock(32, 16, (1, 4, 1), (1, 4, 1)) for _ in range(n_tracks)])
         self.transconv5 = nn.ModuleList([GeneraterBlock(16, 1, (1, 1, 12), (1, 1, 12)) for _ in range(n_tracks)])
     def forward(self, x):
         x = x.view(-1, latent_dim, 1, 1, 1)
         x = self.transconv0(x)
         x = self.transconv1(x)
         x = self.transconv2(x)
         x = self.transconv3(x)
         x = [transconv(x) for transconv in self.transconv4]
         x = torch.cat([transconv(x_) for x_, transconv in zip(x, self.transconv5)], 1)
         x = x.view(-1, n_tracks, n_measures * measure_resolution, n_pitches)
         return x
# Discriminator Block
class DiscriminatorBlock(nn.Module):
     def __init__(self, in_dim, out_dim, kernel, stride):
         super().__init__()
         self.conv = nn.Conv3d(in_dim, out_dim, kernel, stride)
         self.batchnorm = nn.BatchNorm3d(out_dim)
     def forward(self, x):
         x = self.conv(x)
         x = self.batchnorm(x)
         return F.leaky_relu(x)
# Updated Discriminator Model with adjusted kernel and stride sizes
class Discriminator(nn.Module):
      def __init__(self):
          super().__init__()
          # Adjusted kernel and stride sizes to fit the input shape
          self.conv0 = nn.ModuleList([DiscriminatorBlock(1, 16, (1, 1, 3), (1, 1, 1)) for _ in range(n_tracks)])
          self.conv1 = nn.ModuleList([DiscriminatorBlock(16, 16, (1, 3, 1), (1, 2, 1)) for _ in range(n_tracks)])
          self.conv2 = DiscriminatorBlock(16 * n_tracks, 64, (1, 1, 2), (1, 1, 1))
          self.conv3 = DiscriminatorBlock(64, 64, (1, 1, 2), (1, 1, 1))
          self.conv4 = DiscriminatorBlock(64, 128, (1, 3, 1), (1, 2, 1))
          self.conv5 = DiscriminatorBlock(128, 128, (2, 1, 1), (1, 1, 1))
          self.conv6 = DiscriminatorBlock(128, 256, (2, 1, 1), (2, 1, 1))
          self.dense = nn.Linear(256, 1)
      def forward(self, x):
          x = x.view(-1, n_tracks, n_measures, measure_resolution, n_pitches)
          x = [conv(x[:, [i]]) for i, conv in enumerate(self.conv0)]
          x = torch.cat([conv(x_) for x_, conv in zip(x, self.conv1)], 1)
          x = self.conv2(x)
          x = self.conv3(x)
          x = self.conv4(x)
          x = self.conv5(x)
          x = self.conv6(x)
          x = x.view(-1, 256)
          return self.dense(x)

# **Training**

In [ ]:

# Initialize models and move them to GPU
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Optimizers remain the same
g_optimizer = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.9))
d_optimizer = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.9))

# Gradient penalty function
def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, 1, device=device)
    alpha = alpha.expand_as(real_samples)
    interpolates = (alpha * real_samples + (1 - alpha) * fake_samples).requires_grad_(True)
    d_interpolates = discriminator(interpolates)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones_like(d_interpolates, device=device),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

# Training Loop with GPU utilization
n_steps = 5
for epoch in range(n_steps):
    for real_samples in data_loader:
        real_samples = real_samples[0].unsqueeze(1).to(device)
        latent = torch.randn(batch_size, latent_dim, device=device)

        # Multiple discriminator steps
        for _ in range(5):
            d_optimizer.zero_grad()
            prediction_real = discriminator(real_samples)
            d_loss_real = -torch.mean(prediction_real)
            fake_samples = generator(latent).detach()
            prediction_fake_d = discriminator(fake_samples)
            d_loss_fake = torch.mean(prediction_fake_d)
            gradient_penalty = compute_gradient_penalty(discriminator, real_samples, fake_samples)
            d_loss = d_loss_real + d_loss_fake + 10 * gradient_penalty
            d_loss.backward()
            d_optimizer.step()

        # Generator step
        g_optimizer.zero_grad()
        fake_samples = generator(latent)
        prediction_fake_g = discriminator(fake_samples)
        g_loss = -torch.mean(prediction_fake_g)
        g_loss.backward()
        g_optimizer.step()

        print(f"Epoch [{epoch+1}/{n_steps}] - D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


# **Mood configuration of music**

In [ ]:

mp3_output_dir = 'OUTPUT DIRECTORY PATH'
os.makedirs(mp3_output_dir, exist_ok=True)
# Function to configure music based on type/mood
def set_mood_config(mood):
    config = {}
    if mood == 1: # Bittersweet
       config["programs"] = [0, 48, 41] # Piano, Strings, and Bass
       config["track_names"] = ['Soft Piano', 'Emotional Strings', 'Warm Bass']
       config["tempo"] = 50 # Slow, reflective tempo
       config["lowest_pitch"] = 36
       config["allowed_pitches"] = [36, 39, 43, 46, 50, 53, 57, 60] # Minor intervals for bittersweet emotion
       config["velocity_range"] = (35, 50) # Moderate dynamics
       config["note_duration_factor"] = 5.5 # Long notes for smooth transitions
       config["time_step_interval"] = 10
    elif mood == 2: # Uplifting
       config["programs"] = [0, 48, 112] # Piano, Strings, and Bells
       config["track_names"] = ['Bright Piano', 'Lively Strings', 'Gentle Bells']
       config["tempo"] = 120 # Fast tempo for energy and optimism
       config["lowest_pitch"] = 40
       config["allowed_pitches"] = [40, 43, 47, 50, 54, 57, 60, 64, 67]
       config["velocity_range"] = (50, 70) # High dynamics
       config["note_duration_factor"] = 4.0 # Shorter overlapping notes
       config["time_step_interval"] = 6 # Faster transitions
       config["min_duration"] = 20 # Ensure at least 20 seconds of music
    elif mood == 3: # Nature-Inspired
       config["programs"] = [0, 74, 123] # Piano, Flute, and Birds
       config["track_names"] = ['Soft Piano', 'Flute Melody', 'Chirping Birds']
       config["tempo"] = 40 # Slow tempo for grounding
       config["lowest_pitch"] = 32
       config["allowed_pitches"] = [32, 36, 39, 43, 46, 50, 53, 57]
       config["velocity_range"] = (25, 40) # Soft dynamics
       config["note_duration_factor"] = 7.0 # Long atmospheric notes
       config["time_step_interval"] = 12 # Slow rhythm
       config["chirp_probability"] = 0.5 # Higher chirp probability
       config["chirp_overlap_factor"] = 0.15 # More overlapping chirps

    elif mood == 4: # Meditation
       config["programs"] = [0, 89, 96] # Piano, Choir, and Ambient Pad
       config["track_names"] = ['Calm Piano', 'Soothing Choir', 'Meditative Pad']
       config["tempo"] = 30 # Very slow, meditative tempo
       config["lowest_pitch"] = 30
       config["allowed_pitches"] = [30, 33, 36, 40, 43, 47, 50]
       config["velocity_range"] = (20, 35) # Very soft dynamics
       config["note_duration_factor"] = 8.0 # Long, sustained notes
       config["time_step_interval"] = 15 # Minimal rhythm for tranquility
       config["volume_increase"] = 6 # Subtle volume boost for clarity
    return config
# Function to generate and save a single sample based on the selected configuration
def generate_sample(generator, latent_dim, sample_index, config, mood_name):
    device = next(generator.parameters()).device
    z = torch.randn(1, latent_dim,device=device)
    with torch.no_grad():
         sample = generator(z).squeeze(0).cpu().numpy()
    sample = sample[0]
    midi_data = pretty_midi.PrettyMIDI()

    for idx, (program, track_name) in enumerate(zip(config["programs"], config["track_names"])):
        instrument = pretty_midi.Instrument(program=program, is_drum=False, name=track_name)
        for time_step in range(0, sample.shape[1], config["time_step_interval"]):
           for pitch in range(sample.shape[2]):
               if sample[idx, time_step, pitch] > 0.5 and (pitch + config["lowest_pitch"]) in config["allowed_pitches"]:
                  velocity = int(config["velocity_range"][0] + (config["velocity_range"][1] - config["velocity_range"][0]) * torch.rand(1).item())
                  note_start = (time_step * (60 / config["tempo"]) / 4)
                  note_end = note_start + (60 / config["tempo"]) * config["note_duration_factor"]
                  note = pretty_midi.Note(
                  velocity=velocity,
                  pitch=pitch + config["lowest_pitch"],
                  start=note_start,
                  end=note_end
                  )
                  instrument.notes.append(note)
           # Add overlapping random chirps for Nature-Inspired
           if mood_name == "Nature-Inspired" and torch.rand(1).item() < config.get("chirp_probability", 0.5):
              for _ in range(torch.randint(1, 3, (1,)).item()): # Generate 1-3 chirps in each interval
                  chirp_pitch = config["lowest_pitch"] + torch.randint(0, len(config["allowed_pitches"]), (1,)).item()
                  chirp_start = (time_step * (60 / config["tempo"]) / 4) + torch.rand(1).item() * config.get("chirp_overlap_factor", 0.1)
                  chirp_note = pretty_midi.Note(
                  velocity=int(config["velocity_range"][1]), # Slightly louder for chirps
                  pitch=chirp_pitch,
                  start=chirp_start,
                  end=chirp_start + 0.3 # Short chirp
                  )
                  instrument.notes.append(chirp_note)
        midi_data.instruments.append(instrument)
    midi_filename = tempfile.NamedTemporaryFile(suffix=".mid", delete=False).name
    midi_data.write(midi_filename)
    output_wav = f"{mp3_output_dir}/{mood_name}_sample_{sample_index + 1}.wav"
    output_mp3 = f"{mp3_output_dir}/{mood_name}_sample_{sample_index + 1}.mp3"
    sound = AudioSegment.from_wav(output_wav)
    # Increase volume for Meditation without harsh transitions
    if mood_name == "Meditation":
       sound = sound + config.get("volume_increase", 0)
    sound.export(output_mp3, format="mp3")
    os.remove(midi_filename)
    os.remove(output_wav)
    print(f"Generated {mood_name} MP3 file: {output_mp3}")

# **Generating music for selected mood**

In [ ]:
moods = ["Bittersweet", "Uplifting", "Nature-Inspired", "Meditation"]
print("Choose a mood to generate music:")
for idx, mood in enumerate(moods, 1):
    print(f"{idx} - {mood}")
mood_choice = int(input("Enter the number of your choice: "))
config = set_mood_config(mood_choice)
mood_name = moods[mood_choice - 1]
# Generate and save music for the selected mood
generate_sample(generator, latent_dim, i, config, mood_name)